In [64]:
# import bert and pandas
import pandas as pd

# read the data

df_all_years = pd.read_csv('/Users/siddarthkerkar/Documents/airbnb/get the data/london/reviews.csv')
df_all_years.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,13913,80770,2010-08-18,177109,Michael,My girlfriend and I hadn't known Alina before ...
1,13913,367568,2011-07-11,19835707,Mathias,Alina was a really good host. The flat is clea...
2,13913,529579,2011-09-13,1110304,Kristin,Alina is an amazing host. She made me feel rig...
3,13913,595481,2011-10-03,1216358,Camilla,"Alina's place is so nice, the room is big and ..."
4,13913,612947,2011-10-09,490840,Jorik,"Nice location in Islington area, good for shor..."


In [65]:
import pandas as pd

# Load the CSV files into DataFrames
df_jun = pd.read_csv('/Users/siddarthkerkar/Documents/airbnb/get the data/london/jun 2023 reviews.csv')
df_dec = pd.read_csv('/Users/siddarthkerkar/Documents/airbnb/get the data/london/dec 2023.csv')

# Use concat to combine the DataFrames
df_all_years = pd.concat([df_all_years, df_jun, df_dec], ignore_index=True)


In [66]:
# Assume df_all_years is already defined and loaded

# Convert the 'date' column to datetime format if it's not already
df_all_years['date'] = pd.to_datetime(df_all_years['date'])

# Extract the year from the 'date' column
df_all_years['year'] = df_all_years['date'].dt.year

# Filter rows for the year range 2009 to 2023
df_filtered = df_all_years[df_all_years['year'].between(2009, 2023)]

# Group by the 'year' and sample up to 100 entries per group
df_sampled = df_filtered.groupby('year').apply(lambda x: x.sample(min(len(x), 500), random_state=1)).reset_index(drop=True)


In [67]:
df_sampled.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,year
0,9376,15340,2009-10-27,43389,Guillermo & Dana,I stayed at the Venice spot. Everything was gr...,2009
1,15089,13481,2009-10-13,33125,Ana,"We stayed in his apartment for 14 nights, and ...",2009
2,9376,11137,2009-09-23,37125,Stephanie,Cristina's place was a beautiful haven near th...,2009
3,8021,4514,2009-06-24,22533,Chaiwat,The location was perfect and the host was easy...,2009
4,15089,11567,2009-09-27,28381,Will,"great time and awesome place, highly recommend...",2009


In [68]:
from transformers import pipeline

df = df_sampled.copy()

# Load sentiment analysis pipeline
sentiment_model = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# Load emotion classification model
emotion_classifier = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")

# Function to apply sentiment analysis and convert stars to integer
def apply_sentiment_analysis(text):
    if isinstance(text, str):
        # Truncate the input text to the maximum allowed sequence length
        max_length = 512
        truncated_text = text[:max_length]
        
        # Perform sentiment analysis on the truncated text
        result = sentiment_model(truncated_text)
        
        # Extract the first part of the label (e.g., '4' from '4 stars') and convert to integer
        star_rating = int(result[0]['label'].split()[0])
        return star_rating
    else:
        return None  # or a default value, or handle missing values as required

# Function to determine the most prevalent emotion in the text
def detect_emotion(text):
    if isinstance(text, str):
        # You may want to truncate the text here as well if it's very long
        max_length = 512
        truncated_text = text[:max_length]
        
        # Perform emotion detection
        result = emotion_classifier(truncated_text)
        
        # Return the label of the highest scoring emotion
        return result[0]['label']
    else:
        return None  # Handle non-string inputs

# Apply functions to the comments column and save the results
df['sentiment'] = df['comments'].apply(apply_sentiment_analysis)
df['emotion'] = df['comments'].apply(detect_emotion)

# Show the updated DataFrame
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,year,sentiment,emotion
0,9376,15340,2009-10-27,43389,Guillermo & Dana,I stayed at the Venice spot. Everything was gr...,2009,5.0,joy
1,15089,13481,2009-10-13,33125,Ana,"We stayed in his apartment for 14 nights, and ...",2009,5.0,joy
2,9376,11137,2009-09-23,37125,Stephanie,Cristina's place was a beautiful haven near th...,2009,5.0,love
3,8021,4514,2009-06-24,22533,Chaiwat,The location was perfect and the host was easy...,2009,5.0,joy
4,15089,11567,2009-09-27,28381,Will,"great time and awesome place, highly recommend...",2009,5.0,joy


In [69]:
# save df to csv
df.to_csv('/Users/siddarthkerkar/Documents/airbnb/get the data/london/sentiment#2.csv', index=False)


In [70]:
# get the average sentiment scores for each year
df_avg_sentiment = df.groupby('year')['sentiment'].mean().reset_index()
df_avg_sentiment

,year,sentiment
0,2009,4.750000
1,2010,4.688000
2,2011,4.674000
3,2012,4.640000
4,2013,4.644000
5,2014,4.684000
6,2015,4.670000
7,2016,4.648000
8,2017,4.642000
9,2018,4.694000


In [71]:
# graph df_avg_sentiment
# plotly
import plotly.express as px

px.defaults.template = "plotly_dark"
fig = px.line(df_avg_sentiment, x='year', y='sentiment', title='Average Sentiment Score Over Time')
fig.show()

In [72]:
import pandas as pd
import statsmodels.api as sm


df_avg_sentiment

# Define the independent variable (year) and add a constant to the model
X = sm.add_constant(df_avg_sentiment['year'])
y = df_avg_sentiment['sentiment']

# Fit the linear regression model
model = sm.OLS(y, X).fit()

# Get the summary of the regression result
model_summary = model.summary()
model_summary


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/stats/_stats_py.py:1806: UserWarning:

kurtosistest only valid for n>=20 ... continuing anyway, n=15



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              sentiment   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                 -0.043
Method:                 Least Squares   F-statistic:                    0.4243
Date:                Fri, 03 May 2024   Prob (F-statistic):              0.526
Time:                        00:57:36   Log-Likelihood:                 29.085
No. Observations:                  15   AIC:                            -54.17
Df Residuals:                      13   BIC:                            -52.75
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.6087      4.505      1.689      0.115      -2.123      17.341
year          -0.0015      0.002     -0.651      0.526      -0.006       0.003
==============================================================================
Omnibus:                        0.955   Durbin-Watson:                   1.338
Prob(Omnibus):                  0.620   Jarque-Bera (JB):                0.817
Skew:                           0.486   Prob(JB):                        0.665
Kurtosis:                       2.397   Cond. No.                     9.41e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.41e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [77]:
# graph a grouped bar chart for count of emotion by year
df_emotion_counts = df.groupby(['year', 'emotion']).size().unstack().fillna(0)

# Plot the grouped bar chart
px.defaults.template = "plotly_dark"
fig = px.bar(df_emotion_counts, barmode='group', title='Emotion Counts by Year')
fig.show()

In [78]:
# retrive a review that has sentiment of 1 or 2 and anger or fear emotion
df_bad = df[(df['sentiment'].isin([1, 2])) & (df['emotion'].isin(['anger', 'fear']))]
df_bad.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,year,sentiment,emotion
435,5728,35058,2010-04-11,104639,James,They are nice people but next time I'd pitch a...,2010,2.0,anger
696,168348,624849,2011-10-12,120077,Gregor,The host canceled my reservation 15 days befor...,2011,1.0,anger
798,7992,688705,2011-11-03,1352088,Lucas,The host canceled my reservation 10 days befor...,2011,1.0,anger
1047,388030,1048223,2012-03-26,1946373,Keith,Unfortunately I wouldn't recommend Sue as a ho...,2012,2.0,fear
1066,510386,2204328,2012-09-04,2543376,Rodolfo,The reservation was canceled 12 days before ar...,2012,1.0,fear


In [7]:
import pandas as pd

df = pd.read_csv('/Users/siddarthkerkar/Documents/airbnb/get the data/london/sentiment#2.csv')

# Convert the 'date' column to datetime and handle errors by coercing them
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Check if there are any null values after conversion which indicates parsing issues
print(df['date'].isnull().sum())

# Assuming the conversion has no errors or they are acceptable
df['month_year'] = df['date'].dt.to_period('M')

# Group by the new 'month_year' column and the 'year' column, then calculate the mean sentiment score
monthly_sentiment = df.groupby(['month_year', 'year'])['sentiment'].mean().reset_index()

print(monthly_sentiment)


154
    month_year    year  sentiment
0      2009-05  2009.0   4.000000
1      2009-06  2009.0   5.000000
2      2009-07  2009.0   4.666667
3      2009-08  2009.0   4.666667
4      2009-09  2009.0   5.000000
..         ...     ...        ...
170    2023-07  2023.0   4.500000
171    2023-08  2023.0   4.750000
172    2023-09  2023.0   4.487179
173    2023-10  2023.0   4.711111
174    2023-11  2023.0   4.681818

[175 rows x 3 columns]


In [11]:
import plotly.express as px

# Convert 'month_year' from Period to string for compatibility with Plotly
monthly_sentiment['month_year'] = monthly_sentiment['month_year'].astype(str)

# Slice the DataFrame to start from the second row (index 1)
filtered_data = monthly_sentiment.iloc[1:]

# Set the Plotly theme to dark
px.defaults.template = "plotly_dark"

# Create a line plot
fig = px.line(filtered_data, x='month_year', y='sentiment', title='Average Monthly Sentiment Score Over Time')

# Show the figure
fig.show()
